# DESI Iron
Determine how many galaxies were observed in DESI Iron that are good TF targets by finding the number of unique galaxies observed with observations at the center and at .4R.

In [ ]:
## Module imports and constant definitions 
from astropy.table import unique, Table
from astropy.coordinates import SkyCoord
from astropy.io import fits 

import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 

from tqdm.auto import tqdm

import os

import requests

from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle

h = 1
H0 = 100*h
c = 3e5
q0 = 0.2
V0 = 2.5 # 0-point of TFR

### Iron Data
We read in the Iron data to use throughout the notebook. Commented out below is the Fuji data, to use to check against when necessary. 

In [ ]:
tiron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_iron_healpix.fits')
#tiron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_fuji_healpix.fits')

tiron[:5]

### SGA
We read in the SGA to use throughout the notebook. 

In [ ]:
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

## Create a dictionary of SGA IDs to find row indices 
SGA_dict = {}
for i in range(len(SGA)):
    SGA_dict[SGA['SGA_ID'][i]] = i

SGA[:5]

## Separation between Galaxies and Observations
Find all targets on each galaxy, and then calculate distance between SGA center coordinates and observation coordinates. Store the lists of center and axis observations as `iron_centers` and `iron_axis`.

In [ ]:
tiron['SKY_FIBER_DIST'] = 0.
tiron['SKY_FIBER_DIST_R26'] = 0.

## For each SGA galaxy that has 1+ observations, calculate distance for all of its targets
for sga_id in tqdm(np.unique(tiron['SGA_ID'])):
    
    ## Identify all galaxy targets on this galaxy
    obs_idx = tiron['SGA_ID'] == sga_id
    
    ## Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    ## Calculate distance between each observation and the center of the galaxy
    SGA_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                          dec=SGA['DEC'][sga_idx], 
                          unit=u.degree)
    target_coords = SkyCoord(ra=tiron['RA'][obs_idx], 
                             dec=tiron['DEC'][obs_idx], 
                             unit=u.degree)
    sep2d = target_coords.separation(SGA_coords)
    
    ## Add the distance to the tiron table
    tiron['SKY_FIBER_DIST'][obs_idx] = sep2d
    ## Add the distance in R26 to the tiron table 
    tiron['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)

## Isolate the centers to be those measurements where the distance is <.1*R26
centers_boolean = tiron['SKY_FIBER_DIST_R26'] < 0.1

## Sort observations into center and axis observations 
iron_centers = tiron[centers_boolean]
iron_axis = tiron[~centers_boolean]

## Cleaning Iron Center Observations

Only keep observations with
 * `DELTACHI2` > 25
 * `ZWARN` == 0

In [ ]:
good_centers = iron_centers[(iron_centers['DELTACHI2'] > 25) & (iron_centers['ZWARN'] == 0)]

## Check for multiple good center observations 
unique_ids, counts = np.unique(good_centers['SGA_ID'], return_counts=True)

If there is at least one good center observation, set the galaxy's redshift as the weighted average of all observation redshifts.

In [ ]:
SGA['Z_DESI'] = np.nan
SGA['ZERR_DESI'] = np.nan

weights = 1./(good_centers['ZERR']**2)

for sga_id in np.unique(good_centers['SGA_ID']):
    
    ## Find all the center observations of this galaxy
    obs_idx = good_centers['SGA_ID'] == sga_id
    
    ## Find the row in SGA for this galaxy
    SGA_idx = SGA_dict[sga_id]
    
    # Set the redshift of this galaxy to be weighted average of all good center observation redshifts
    SGA['Z_DESI'][SGA_idx] = np.average(good_centers['Z'][obs_idx], 
                                        weights=weights[obs_idx])
    SGA['ZERR_DESI'][SGA_idx] = np.sqrt(1./np.sum(weights[obs_idx]))

Determine how many observations we have at $.4R$ that have a corresponding center observation for that galaxy.

In [ ]:
## Include all observations where .38R < distance < .42R in our .4R values
r0p4 = iron_axis[(iron_axis['SKY_FIBER_DIST_R26'] > 0.38) & (iron_axis['SKY_FIBER_DIST_R26'] < 0.42)]

## Count how many of these are unique values
unique_centers = np.unique(good_centers['SGA_ID'])
unique_r0p4 = np.unique(r0p4['SGA_ID'])

centers_and_p4s = []
for i in unique_r0p4: 
    if i in unique_centers: 
        centers_and_p4s.append(i)

## Output the number of observations that are generally good for TF fitting
print(len(centers_and_p4s)," unique galaxies with center and .4R observations")

## Find the percentage of all Iron galaxies that are generally good for TF fitting
num_iron = len(np.unique(tiron['SGA_ID']))
percentage_TF = 100*((len(centers_and_p4s))/num_iron)
print(percentage_TF,"% of all Iron galaxies")

## Cluster Membership
Following Cosmicflows4 (Kourkchi et al. 2020), cluster membership is defined as
- $R_p < 1.5R_{2t}$ and $v < V_c \pm 3\sigma_p$
- $1.5R_{2t} \leq R_p < 3R_{2t}$ and $v < V_c \pm 2\sigma_p$

where $R_p$ is the projected distance from the cluster center, $R_{2t}$ is the cluster projected second turnaround radius, $\sigma_p$ is the projected velocity dispersion of the cluster, and $V_c$ is the average heliocentric radial velocity of the cluster.

### Tully et. al. (2015) Table 3

In [ ]:
hdu = fits.open("../Tully15-Table3.fits")
table3 = Table(hdu[1].data)
table3.rename_column("<Vcmba>","VMod")
hdu.close()

table3_dict = {}
for i in range(len(table3)):
    table3_dict[table3['Nest'][i]] = i

table3[:5]

In [ ]:
## Check a suspicious value from later on in the notebook
table3[table3['Nest'] == 100526]

In [ ]:
## Continue the suspicious value check 
cluster_coord = SkyCoord(table3['SGLON'][table3['Nest'] == 100526]*u.degree, 
                         table3['SGLAT'][table3['Nest'] == 100526]*u.degree, 
                         frame='supergalactic')
print(cluster_coord.transform_to('icrs'))

### Tully et. al (2013) Table 2

In [ ]:
hdu = fits.open("../Tully13-Table2.fit")
table2 = Table(hdu[1].data)
hdu.close()

table2[:5]

Loop through the SGA table and collect just the rows with `SGA_ID` values that are included in `centers_and_p4s` in a new table.

In [ ]:
SGA.add_index("SGA_ID")
TF_SGA_cp4 = SGA.loc[centers_and_p4s]

TF_SGA_cp4[:5]

## Determine Iron Cluster Membership
This cell loops through all Nest numbers given in Tully 2015 Table 3, determines the coordinates, and matches galaxies to their nearest cluster. It collects the center and axis observations for each cluster, and stores the Nest number in `full_clusters` along with a table of center and axis observations in `c_4r_in_cluster`. 

At present, this cell is returning some suspicious values.

In [ ]:
full_cluster = [] 
c_4r_in_cluster = []

for nest_id in tqdm(table3["Nest"]):
    ## Set active row for each nest number and base values 
    active_row = table3_dict[nest_id]
    R2t = table3["R2t"][active_row]
    sigma = table3["sigP"][active_row]
    
    ## Find the coordinates for each cluster
    cluster_coords = SkyCoord(table3["SGLON"][active_row]*u.degree, 
                       table3["SGLAT"][active_row]*u.degree, 
                       frame='supergalactic')
    
    group_coords = SkyCoord(table2['SGLON']*u.degree, 
                        table2['SGLAT']*u.degree, 
                        frame='supergalactic')
    
    ## Match cluster to group coordinates 
    idx, d2d, d3d = cluster_coords.match_to_catalog_sky(group_coords)
    v = table2["__HV_"][idx]
    
    ## Match SGA coordinates from unique galaxies to nearest cluster
    SGA_coords = SkyCoord(TF_SGA_cp4['RA'], TF_SGA_cp4['DEC'], unit='deg')
    sep = cluster_coords.separation(SGA_coords)

    ## Convert R2t to an angle 
    R2t_angle = (R2t/(v/H0))*u.radian
    
    SGA_in_cluster1 = (sep < 1.5*R2t_angle) & (TF_SGA_cp4["Z_DESI"]*c > v - 3*sigma) & (TF_SGA_cp4["Z_DESI"]*c < v + 3*sigma)
    SGA_in_cluster2 = (sep >= 1.5*R2t_angle) & (sep < 3*R2t_angle) & (TF_SGA_cp4["Z_DESI"]*c > v - 2*sigma) & (TF_SGA_cp4["Z_DESI"]*c < v + 2*sigma)
    
    SGA_in_cluster = SGA_in_cluster1 | SGA_in_cluster2
    
    ## Keep observations that are within nest cluster 
    SGA_ID_in_cluster = TF_SGA_cp4["SGA_ID"][SGA_in_cluster]
    
    ## Gather centers and axes in the cluster
    centers_in_cluster = good_centers[np.in1d(good_centers['SGA_ID'], SGA_ID_in_cluster)]
    axis_in_cluster = iron_axis[np.in1d(iron_axis['SGA_ID'], SGA_ID_in_cluster)]
    
    if len(SGA_ID_in_cluster) > 20:
        print("CLUSTER ID:", nest_id, "--", len(SGA_ID_in_cluster), "GALAXIES."),
        full_cluster.append(nest_id)
        c_4r_in_cluster.append(SGA_ID_in_cluster)